# 🔬 Research Proposal Agentic System for High School Science Competitions

This notebook demonstrates a multi-agent system that collaboratively generates research proposals for high school science competitions such as Synopsis.

## System Overview

The system consists of 7 specialized agents working in a coordinated workflow:

1. **WinningProjectsResearcher** - Searches for and compiles winning projects from recent competitions
2. **CriteriaResearcher** - Researches evaluation criteria and rubrics from major competitions
3. **CriteriaAnalyzer** - Correlates winning projects with criteria and creates practical guidelines
4. **TopicIntersector** - Identifies topics that have won competitions AND are currently active research areas
5. **TopicProposer** - Selects the best topic and specifies evaluation criteria
6. **ProposalWriter** - Writes a comprehensive research proposal
7. **ProposalEvaluator** - Evaluates the proposal and suggests improvements

## Architecture

The workflow uses a combination of parallel, sequential, and loop patterns to efficiently generate winning proposals.


## 📖 Setup

### Configure your Gemini API Key

This notebook uses the Gemini API, which requires authentication.

**1. Get your API key**

If you don't have one already, create an [API key in Google AI Studio](https://aistudio.google.com/app/api-keys).

**2. Add the key to Kaggle Secrets (if using Kaggle)**

1. In the top menu bar, select `Add-ons` then `Secrets`.
2. Create a new secret with the label `GOOGLE_API_KEY`.
3. Paste your API key and click "Save".

**3. Or use environment variables**

Set `GOOGLE_API_KEY` in your environment or `.env` file.


In [ ]:
import os
from dotenv import load_dotenv

# Try Kaggle secrets first (for Kaggle notebooks)
try:
    from kaggle_secrets import UserSecretsClient
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    print("✅ Gemini API key setup complete (from Kaggle secrets).")
except:
    # Fall back to environment variables or .env file
    load_dotenv()
    if os.getenv("GOOGLE_API_KEY"):
        print("✅ Gemini API key setup complete (from environment/.env).")
    else:
        print("⚠️  Warning: GOOGLE_API_KEY not found. Please set it in your environment or .env file.")


### Import Required Libraries


In [ ]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import AgentTool, FunctionTool, google_search, ToolContext
from google.genai import types
import asyncio

print("✅ ADK components imported successfully.")


## 🛠️ Custom Tools

We'll create custom tools that agents can use to enhance their capabilities.


In [ ]:
def exit_proposal_loop():
    """
    Call this function when the proposal evaluation indicates approval.
    This exits the iterative refinement loop.
    """
    return {"status": "approved", "message": "Proposal approved. Exiting refinement loop."}

print("✅ Custom tools defined.")


## 👥 Agent Definitions

Let's define our team of specialized agents.


### Agent 1: WinningProjectsResearcher

This agent searches for and compiles information about winning projects from recent high school science competitions.


In [ ]:
winning_projects_researcher = Agent(
    name="WinningProjectsResearcher",
    model="gemini-2.5-flash-lite",
    instruction="""You are a research specialist focused on finding winning high school science competition projects.

Your task is to search for and compile information about projects that have won major high school science competitions 
(such as Synopsis, Regeneron Science Talent Search, Intel ISEF, Google Science Fair, etc.) in recent years (2020-2024).

For each winning project you find, extract:
- Project title
- Brief description (2-3 sentences)
- Competition name and year
- Key research area/topic

Present your findings as a structured list with at least 8-10 winning projects across different scientific disciplines.
Focus on projects that demonstrate:
- Real-world relevance and societal impact
- Scientific rigor appropriate for high school students
- Innovation and creativity

Use the google_search tool to find this information.""",
    tools=[google_search],
    output_key="winning_projects",
)

print("✅ WinningProjectsResearcher created.")


### Agent 2: CriteriaResearcher

This agent researches evaluation criteria and rubrics from major competitions.


In [ ]:
criteria_researcher = Agent(
    name="CriteriaResearcher",
    model="gemini-2.5-flash-lite",
    instruction="""You are a specialist in understanding evaluation criteria for high school science competitions.

Your task is to search for and compile the official criteria and rubrics used to evaluate projects in major 
high school science competitions (Synopsis, Regeneron STS, Intel ISEF, Google Science Fair, etc.).

For each competition, extract:
- Evaluation criteria categories (e.g., Scientific Method, Innovation, Impact, Presentation)
- Scoring rubrics or point allocations
- Key factors that judges look for
- Common reasons projects win or lose

Present your findings in a structured format that clearly shows:
- What criteria are most important
- How projects are scored
- What distinguishes winning projects

Use the google_search tool to find this information.""",
    tools=[google_search],
    output_key="evaluation_criteria",
)

print("✅ CriteriaResearcher created.")


### Agent 3: CriteriaAnalyzer

This agent correlates winning projects with evaluation criteria and creates practical guidelines.


In [ ]:
criteria_analyzer = Agent(
    name="CriteriaAnalyzer",
    model="gemini-2.5-flash-lite",
    instruction="""You are an expert analyst who correlates winning projects with evaluation criteria.

Your task is to analyze the winning projects and evaluation criteria provided, and create a practical, 
actionable set of criteria and rubrics that can guide the generation of a winning topic and project proposal.

Inputs:
- Winning Projects: {winning_projects}
- Evaluation Criteria: {evaluation_criteria}

Analyze:
1. What common characteristics do winning projects share?
2. How do these characteristics align with the evaluation criteria?
3. What patterns emerge across different competitions?
4. What are the critical success factors?

Output a comprehensive, practical guide that includes:
- A prioritized list of evaluation criteria (most important first)
- Specific rubrics or checklists for each criterion
- Key characteristics that winning projects demonstrate
- Actionable guidelines for topic selection
- Actionable guidelines for proposal writing

Format your output clearly with sections and bullet points.""",
    output_key="practical_criteria",
)

print("✅ CriteriaAnalyzer created.")


### Agent 4: TopicIntersector

This agent identifies topics that have won competitions AND are currently active research areas.


In [ ]:
topic_intersector = Agent(
    name="TopicIntersector",
    model="gemini-2.5-flash-lite",
    instruction="""You are a research strategist who identifies topics that are both:
1. Proven winners in high school competitions
2. Currently of active interest to the research community

Your task is to:
1. Extract the main topic areas from the winning projects: {winning_projects}
2. For each topic area, search for current active research using google_search
3. Identify 3-5 topics that meet BOTH criteria:
   - Have won competitions (proven track record)
   - Are currently active areas of research (2024-2025)

For each selected topic, provide:
- Topic name and brief description
- Evidence it has won competitions (cite specific projects)
- Evidence of current research activity
- Why it's suitable for high school students (accessibility)
- Why it matters to society/humanity

Use google_search to verify current research activity for each topic.""",
    tools=[google_search],
    output_key="intersected_topics",
)

print("✅ TopicIntersector created.")


### Agent 5: TopicProposer

This agent selects the best topic and specifies evaluation criteria.


In [ ]:
topic_proposer = Agent(
    name="TopicProposer",
    model="gemini-2.5-flash-lite",
    instruction="""You are a topic selection expert who proposes the best research topic for a high school science competition.

Based on the following inputs:
- Practical Criteria: {practical_criteria}
- Intersected Topics: {intersected_topics}

Your task is to:
1. Select the BEST single topic from the intersected topics that:
   - Best aligns with the practical criteria
   - Has the highest potential for winning
   - Is most accessible to high school students
   - Has strong societal/research community interest

2. Specify the evaluation criteria and rubric that will be used to evaluate a project with this topic:
   - Adapt the practical criteria to this specific topic
   - Create a detailed rubric with scoring guidelines
   - Identify key success factors for this topic

Output your proposal in this format:

**SELECTED TOPIC:**
[Topic name and description]

**WHY THIS TOPIC:**
- Alignment with winning criteria: [explanation]
- Accessibility for high school students: [explanation]
- Current research interest: [explanation]
- Societal impact: [explanation]

**EVALUATION CRITERIA AND RUBRIC:**
[Detailed criteria and scoring rubric specific to this topic]

**KEY SUCCESS FACTORS:**
[List of critical factors that will determine success]""",
    output_key="topic_proposal",
)

print("✅ TopicProposer created.")


### Agent 6: Initial ProposalWriter

This agent writes the initial research proposal (first iteration).


In [ ]:
initial_proposal_writer = Agent(
    name="InitialProposalWriter",
    model="gemini-2.5-flash-lite",
    instruction="""You are an expert proposal writer for high school science competitions.

Your task is to write a comprehensive research proposal based on:
- Selected Topic: {topic_proposal}

Write a complete research proposal that includes:

1. **Title**: Clear, descriptive, and engaging
2. **Abstract/Summary**: Brief overview (150-200 words)
3. **Introduction & Background**: 
   - Problem statement
   - Why this research matters (societal, community, research community)
   - Current state of knowledge
4. **Research Objectives**: Clear, specific, measurable objectives
5. **Methodology**: 
   - Detailed research plan
   - Methods appropriate for high school students
   - Timeline for one semester
   - Resources needed
6. **Expected Outcomes & Impact**:
   - What will be learned/discovered
   - How it benefits society/humanity
   - Contribution to research community
7. **Feasibility**: 
   - Why this is achievable in one semester
   - Student capabilities required
   - Risk assessment and mitigation

The proposal should be:
- Well-structured and professional
- Aligned with the evaluation criteria and rubric specified in the topic proposal
- Appropriate for high school level
- Compelling and likely to win""",
    output_key="research_proposal",
)

print("✅ InitialProposalWriter created.")


### Agent 7: Refined ProposalWriter

This agent revises the proposal based on evaluation feedback. It can exit the loop when approved.


In [ ]:
refined_proposal_writer = Agent(
    name="RefinedProposalWriter",
    model="gemini-2.5-flash-lite",
    instruction="""You are an expert proposal writer for high school science competitions.

Your task is to revise the research proposal based on:
- Selected Topic: {topic_proposal}
- Current Proposal: {research_proposal}
- Evaluation Feedback (from the evaluator): {evaluation_feedback}

IMPORTANT: 
- Check the evaluation feedback. If it is EXACTLY "APPROVED", you MUST call the `exit_proposal_loop` function.
- Otherwise, revise the proposal to address all feedback points while maintaining all required sections:
  1. Title
  2. Abstract/Summary (150-200 words)
  3. Introduction & Background
  4. Research Objectives
  5. Methodology
  6. Expected Outcomes & Impact
  7. Feasibility

The revised proposal should be well-structured, aligned with criteria, and compelling.""",
    output_key="research_proposal",
    tools=[FunctionTool(exit_proposal_loop)],
)

print("✅ RefinedProposalWriter created.")


### Agent 8: Initial ProposalEvaluator

This agent evaluates the initial proposal and creates evaluation feedback.


In [ ]:
initial_proposal_evaluator = Agent(
    name="InitialProposalEvaluator",
    model="gemini-2.5-flash-lite",
    instruction="""You are a strict but fair evaluator of high school science research proposals.

Your task is to evaluate the research proposal against the criteria and rubric specified in the topic proposal.

Inputs:
- Research Proposal: {research_proposal}
- Topic Proposal (contains criteria): {topic_proposal}

Evaluate the proposal on:
1. Alignment with evaluation criteria
2. Scientific rigor and methodology
3. Feasibility for high school students
4. Potential impact and significance
5. Clarity and presentation quality
6. Likelihood of winning

Provide:
- Overall assessment (score/rating)
- Strengths of the proposal
- Specific weaknesses or gaps
- Actionable suggestions for improvement

If the proposal meets all criteria and has a high likelihood of winning, respond with EXACTLY: "APPROVED"
Otherwise, provide detailed, specific feedback for improvement.""",
    output_key="evaluation_feedback",
)

print("✅ InitialProposalEvaluator created.")


### Agent 9: Refinement ProposalEvaluator

This agent evaluates revised proposals during the refinement loop.


In [ ]:
refinement_proposal_evaluator = Agent(
    name="RefinementProposalEvaluator",
    model="gemini-2.5-flash-lite",
    instruction="""You are a strict but fair evaluator of high school science research proposals.

Your task is to evaluate the research proposal against the criteria and rubric specified in the topic proposal.

Inputs:
- Research Proposal: {research_proposal}
- Topic Proposal (contains criteria): {topic_proposal}

Evaluate the proposal on:
1. Alignment with evaluation criteria
2. Scientific rigor and methodology
3. Feasibility for high school students
4. Potential impact and significance
5. Clarity and presentation quality
6. Likelihood of winning

Provide:
- Overall assessment (score/rating)
- Strengths of the proposal
- Specific weaknesses or gaps
- Actionable suggestions for improvement

If the proposal meets all criteria and has a high likelihood of winning, respond with EXACTLY: "APPROVED"
Otherwise, provide detailed, specific feedback for improvement.""",
    output_key="evaluation_feedback",
)

print("✅ RefinementProposalEvaluator created.")


## 🔄 Workflow Orchestration

Now we'll orchestrate the agents using parallel, sequential, and loop patterns.


### Step 1: Parallel Research Phase

Both researchers work simultaneously for efficiency.


In [ ]:
parallel_research_phase = ParallelAgent(
    name="ParallelResearchPhase",
    sub_agents=[winning_projects_researcher, criteria_researcher],
)

print("✅ ParallelResearchPhase created.")


### Step 2: Analysis Phase

Analyze criteria after research is complete.


In [ ]:
analysis_phase = SequentialAgent(
    name="AnalysisPhase",
    sub_agents=[criteria_analyzer],
)

print("✅ AnalysisPhase created.")


### Step 3: Topic Intersection Phase

Find topics that meet both criteria (winning + current research).


In [ ]:
topic_intersection_phase = SequentialAgent(
    name="TopicIntersectionPhase",
    sub_agents=[topic_intersector],
)

print("✅ TopicIntersectionPhase created.")


### Step 4: Topic Proposal Phase

Select the best topic with evaluation criteria.


In [ ]:
topic_proposal_phase = SequentialAgent(
    name="TopicProposalPhase",
    sub_agents=[topic_proposer],
)

print("✅ TopicProposalPhase created.")


### Step 5: Initial Proposal Phase

Write the initial research proposal.


In [ ]:
initial_proposal_phase = SequentialAgent(
    name="InitialProposalPhase",
    sub_agents=[initial_proposal_writer],
)

print("✅ InitialProposalPhase created.")


### Step 6: Initial Evaluation Phase

Evaluate the initial proposal to create evaluation feedback.


In [ ]:
initial_evaluation_phase = SequentialAgent(
    name="InitialEvaluationPhase",
    sub_agents=[initial_proposal_evaluator],
)

print("✅ InitialEvaluationPhase created.")


### Step 7: Proposal Refinement Loop

Iteratively improve the proposal until it's approved.


In [ ]:
proposal_refinement_loop = LoopAgent(
    name="ProposalRefinementLoop",
    sub_agents=[refined_proposal_writer, refinement_proposal_evaluator],
    max_iterations=3,  # Limit iterations to prevent infinite loops
)

print("✅ ProposalRefinementLoop created.")


### Step 8: Proposal Development Phase

Combine initial proposal, evaluation, and refinement.


In [ ]:
proposal_development_phase = SequentialAgent(
    name="ProposalDevelopmentPhase",
    sub_agents=[
        initial_proposal_phase,      # Write initial proposal
        initial_evaluation_phase,    # Evaluate it (creates evaluation_feedback)
        proposal_refinement_loop,    # Refine iteratively
    ],
)

print("✅ ProposalDevelopmentPhase created.")


### Root Agent: Complete Workflow

Orchestrate all phases in sequence.


In [ ]:
root_agent = SequentialAgent(
    name="ResearchProposalSystem",
    sub_agents=[
        parallel_research_phase,      # Step 1: Parallel research
        analysis_phase,                # Step 2: Analyze criteria
        topic_intersection_phase,      # Step 3: Intersect topics
        topic_proposal_phase,          # Step 4: Propose topic
        proposal_development_phase,    # Step 5: Write initial proposal and refine
    ],
)

print("✅ Root agent (ResearchProposalSystem) created.")
print("\n🎉 All agents and workflow orchestration complete!")


## 🚀 Running the System

Now let's run the complete system to generate a research proposal.


In [ ]:
async def run_research_proposal_system():
    """
    Run the complete research proposal generation system.
    """
    # Initialize session service
    session_service = InMemorySessionService()
    app_name = "ResearchProposalSystem"
    user_id = "student_researcher"
    session_id = "research_session"
    
    # Create runner with session service
    runner = Runner(
        agent=root_agent,
        app_name=app_name,
        session_service=session_service,
    )
    
    # Create a session
    session = await session_service.create_session(
        app_name=app_name,
        user_id=user_id,
        session_id=session_id,
    )
    
    print("=" * 80)
    print("🔬 Research Proposal Agentic System for High School Science Competitions")
    print("=" * 80)
    print("\nThis system will:")
    print("1. Research winning projects and evaluation criteria")
    print("2. Analyze and correlate criteria with winning projects")
    print("3. Intersect winning topics with current research")
    print("4. Propose a topic with evaluation criteria")
    print("5. Write and iteratively refine a research proposal")
    print("\n" + "=" * 80 + "\n")
    
    # User query
    user_query = """Generate a research proposal for a high school science competition 
    such as Synopsis. The proposal should be for a one-semester research project that:
    - Is accessible to high school students
    - Is of interest to society, humanity, community, and research community
    - Is an area of current active research
    - Has a good chance of winning based on competition criteria"""
    
    print(f"📝 User Query:\n{user_query}\n")
    print("=" * 80)
    print("🚀 Starting agentic workflow...\n")
    
    # Convert query to Content format
    user_content = types.Content(
        role="user",
        parts=[types.Part(text=user_query)]
    )
    
    # Run the agent
    async for event in runner.run_async(
        user_id=user_id,
        session_id=session.id,
        new_message=user_content,
    ):
        # Process events if needed
        pass
    
    # Get the final session to access state
    final_session = await session_service.get_session(
        app_name=app_name,
        user_id=user_id,
        session_id=session_id,
    )
    
    # Display results
    print("\n" + "=" * 80)
    print("✅ Workflow Complete!")
    print("=" * 80)
    
    # Extract and display key outputs from session state
    state = final_session.state
    
    print("\n📊 Key Outputs:\n")
    
    if "winning_projects" in state:
        print("1️⃣  WINNING PROJECTS RESEARCH:")
        print("-" * 80)
        projects = state["winning_projects"]
        print(projects[:500] + "..." if len(projects) > 500 else projects)
        print()
    
    if "evaluation_criteria" in state:
        print("2️⃣  EVALUATION CRITERIA RESEARCH:")
        print("-" * 80)
        criteria = state["evaluation_criteria"]
        print(criteria[:500] + "..." if len(criteria) > 500 else criteria)
        print()
    
    if "practical_criteria" in state:
        print("3️⃣  PRACTICAL CRITERIA ANALYSIS:")
        print("-" * 80)
        practical = state["practical_criteria"]
        print(practical[:500] + "..." if len(practical) > 500 else practical)
        print()
    
    if "intersected_topics" in state:
        print("4️⃣  INTERSECTED TOPICS:")
        print("-" * 80)
        topics = state["intersected_topics"]
        print(topics[:500] + "..." if len(topics) > 500 else topics)
        print()
    
    if "topic_proposal" in state:
        print("5️⃣  SELECTED TOPIC PROPOSAL:")
        print("=" * 80)
        print(state["topic_proposal"])
        print()
    
    if "research_proposal" in state:
        print("6️⃣  FINAL RESEARCH PROPOSAL:")
        print("=" * 80)
        print(state["research_proposal"])
        print()
    
    if "evaluation_feedback" in state:
        print("7️⃣  EVALUATION FEEDBACK:")
        print("-" * 80)
        print(state["evaluation_feedback"])
        print()
    
    print("=" * 80)
    print("✨ Process Complete!")
    print("=" * 80)
    
    return final_session

print("✅ Run function defined. Execute the next cell to run the system.")


In [ ]:
# Run the system
await run_research_proposal_system()
